### Modelagem e Previsão de Custos Médicos: Regressão Não Linear Múltipla com Dummies para Análise de Variáveis Qualitativas e Quantitativas

Neste projeto de Data Science, meu objetivo é entender e prever os custos médicos das pessoas com base em algumas variáveis. Tenho dados de 200 pessoas que passaram por consultas ou procedimentos médicos, e quero descobrir quanto elas gastaram com despesas médicas. Para isso, vou analisar várias coisas: a idade das pessoas, quantas doenças crônicas elas têm, a renda delas e o plano de saúde que possuem. Usarei uma técnica chamada regressão não linear múltipla, que me ajudará a encontrar relações complexas entre essas variáveis e os gastos médicos. Além disso, utilizarei "dummies", que são uma forma de incluir variáveis qualitativas, como o tipo de plano de saúde, na minha análise. No final, espero obter insights valiosos que me ajudem a entender melhor esses gastos e até prever quanto uma pessoa pode gastar, dependendo das suas características.

In [4]:
# Instalação dos pacotes

# !pip install pandas
# !pip install numpy
# !pip install -U seaborn
# !pip install matplotlib
# !pip install plotly
# !pip install scipy
# !pip install statsmodels
# !pip install scikit-learn
# !pip install pingouin
# !pip install statstests

In [7]:
# Importação dos pacotes

import pandas as pd # manipulação de dados em formato de dataframe
import numpy as np # operações matemáticas
import seaborn as sns # visualização gráfica
import matplotlib.pyplot as plt # visualização gráfica
import plotly.graph_objects as go # gráficos 3D
from scipy.stats import pearsonr # correlações de Pearson
import statsmodels.api as sm # estimação de modelos
from statsmodels.iolib.summary2 import summary_col # comparação entre modelos
from sklearn.preprocessing import LabelEncoder # transformação de dados
import pingouin as pg # outro modo para obtenção de matrizes de correlações
from statstests.process import stepwise # procedimento Stepwise
from statstests.tests import shapiro_francia # teste de Shapiro-Francia
from scipy.stats import boxcox # transformação de Box-Cox
from scipy.stats import norm # para plotagem da curva normal
from scipy import stats # utilizado na definição da função 'breusch_pagan_test'

In [8]:
# Carregando a base de dados
df_planosaude = pd.read_csv('planosaude.csv', delimiter=',')
df_planosaude

,id,despmed,idade,dcron,renda,plano
0,1,206.84,57,5,57.2,bronze
1,2,184.19,67,4,67.2,bronze
2,3,175.68,62,4,62.0,bronze
3,4,180.04,67,5,67.2,bronze
4,5,218.86,68,5,68.0,bronze
...,...,...,...,...,...,...
195,196,159.02,63,3,63.2,esmeralda
196,197,205.35,64,4,64.0,bronze
197,198,133.64,49,4,48.8,esmeralda
198,199,158.73,44,2,44.0,esmeralda


In [ ]:
# Características das variáveis do dataset
df_planosaude.info()